In [1]:
import numpy as np
import pandas as pd

# 忽略警告提示
import warnings
warnings.filterwarnings('ignore')

In [2]:
#导入数据
train = pd.read_csv("../input/titanic/train.csv")
test  = pd.read_csv("../input/titanic/test.csv")

print ('train:',train.shape)
print('test:',test.shape)

In [3]:
#合并数据集，这样就只需处理一次数据,不合并的话要处理两次
all = train.append( test , ignore_index = True )

print ('all:',all.shape)

In [4]:
all.head()

In [5]:
all.describe()

In [6]:
'''
一共有1309行数据
其中
年龄（Age）        一共1046条，缺失了263条
船票价格（Fare）    一共1308条，缺失了1条
登船港口（Embarked）一共1307条，缺失了2条
船舱号（Cabin）     一共295条，缺失了1014条
'''
all.info()

In [7]:
#数据清洗
#此处用年龄平均值填充年龄缺失值
all['Age'] = all['Age'].fillna(all['Age'].mean())
#此处用船票价格平均值填充船票价格缺失值
all['Fare'] = all['Fare'].fillna(all['Fare'].mean())

In [8]:
#观察登船港口数据的分布情况
all['Embarked'].value_counts()

In [9]:
#我们发现大部分乘客的登船港口都是S,又因为'Embarked'数据只缺失了2条,所以这里直接用出现次数最多的值填充缺失值
all['Embarked'] = all['Embarked'].fillna('S')

In [10]:
#船舱号由于缺失过多,这里选择直接删除船舱号'Cabin'列
del all['Cabin']

In [11]:
#查看处理后的数据
all.shape

In [12]:
all.head()

In [13]:
all.info()

In [14]:
#特征提取
#数据分类

In [15]:
#观察性别数据
all['Sex'].head()

In [16]:
#将性别数据的值映射为数值,男（male）对应数值1，女（female）对应数值0
sex_mapDict={'male':1,'female':0}
all['Sex']=all['Sex'].map(sex_mapDict)
#注:因为性别数据已经改为0-1分布,所以无需进行one-hot编码产生虚拟变量
#观察修改后的性别数据
all['Sex'].head()

In [17]:
#观察登船港口数据
all['Embarked'].head()

In [18]:
#使用get_dummies进行one-hot编码，产生虚拟变量
embarkedDummy = pd.DataFrame()
embarkedDummy = pd.get_dummies( all['Embarked'] , prefix='Embarked' )
embarkedDummy.head()

In [19]:
#添加虚拟变量embarkedDummy到数据集all,并删除原Embarked数据
all = pd.concat([all,embarkedDummy],axis=1)
del all['Embarked']
#观察修改后的数据
all.head()

In [20]:
#观察客舱等级数据
train['Pclass'].head()

In [21]:
#使用get_dummies进行one-hot编码，产生虚拟变量
pclassDummy = pd.DataFrame()
pclassDummy = pd.get_dummies( all['Pclass'] , prefix='Pclass' )
pclassDummy.head()

In [22]:
#添加虚拟变量pclassDummy到数据集all,并删除原Pclass数据
all = pd.concat([all,pclassDummy],axis=1)
del all['Pclass']
#观察修改后的数据
all.head()

In [23]:
#观察姓名数据
all['Name'].head()

In [24]:
#由于姓名数据比较复杂,这里选择不分析此数据而直接删除
del all['Name']
#观察修改后的数据
all.head()

In [25]:
#创建"船上家庭人数"数据，此后简称为家庭
familyDummy = pd.DataFrame()
familyDummy['ShipFamily'] = all['Parch'] + all['SibSp'] + 1

'''
家庭类别：
小家庭(独自一人)ShipFamily_Small：人数=1
中等家庭ShipFamily_Medium: 2<=人数<=3
大家庭ShipFamily_Large: 人数>=4
'''
#if 条件为真时返回if内容，否则返回0
familyDummy['Family_Small'] = familyDummy['ShipFamily'].map( lambda s : 1 if s==1 else 0 )
familyDummy['Family_Medium']  = familyDummy['ShipFamily'].map( lambda s : 1 if 2<=s<=3 else 0 )
familyDummy['Family_Large']  = familyDummy['ShipFamily'].map( lambda s : 1 if s>=4 else 0 )

familyDummy.head()

In [26]:
#添加虚拟变量familyDummy到数据集all,此处生成了船上家庭人数,和小、中、大三种家庭大小，共计四个属性
all = pd.concat([all,familyDummy],axis=1)

#观察修改后的数据
all.head()

In [27]:
all.shape

In [28]:
all.info()

In [29]:
#特征工程
#相关性矩阵
corrDf = all.corr() 
corrDf

In [30]:
#查看各个特征与生存情况（Survived）的相关系数
corrDf['Survived'].sort_values(ascending=False)

In [31]:
#此处选择我创建好的虚拟变量，以及一些相关系数绝对值较大的特征作为模型的输入
X_all = pd.concat( [
                     pclassDummy,#客舱等级
                     familyDummy,#家庭大小
                     embarkedDummy,#登船港口
                     all['Sex'],#性别
                     all['Age'],#年龄
                     all['Fare'],#船票价格
                    ] , axis=1 )
X_all.head()

In [32]:
#模型构建
#由于kaggle给的test.csv文件中并没有“Survived”属性,故无法用于模型训练，也无法用于检测模型训练情况。
#也就是说，我们需要划分kaggle给的train.csv文件，一部分数据用于训练，剩余部分用于本地测试
#此处我们将kaggle给的train.csv数据作为原始数据集“source”，test.csv数据作为需要预测的数据集“forecast”
sourceRow=891  #原始数据集的行数
X_source = X_all.loc[0:sourceRow-1,:]  #原始数据集：我选择的特征
y_source = all.loc[0:sourceRow-1,'Survived']   #原始数据集：生存情况标签

In [33]:
#分割训练集和测试集
from sklearn.model_selection import train_test_split

#分割训练集和测试集,此处训练集占总样本的80%
X_train, X_test, y_train, y_test = train_test_split(X_source,y_source,train_size=0.8,random_state=0)

In [34]:
#模型训练
#此处选择机器学习算法中的随机森林Random Forest算法进行模型训练
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [35]:
#使用十折交叉验证获取max_depth（子树的最大深度）的最优取值(可能),获取范围:1-8
d_scores = []
for i in range(1,9):
    model = RandomForestClassifier(n_estimators=20,max_depth = i,criterion='entropy')
    scores = cross_val_score(model, X_source, y_source, cv=10, scoring='accuracy')
    d_scores.append(scores.mean())
print('max_depth分别取1，2，3，4，5, 6, 7, 8时得到的准确率:')
print(d_scores)
print('准确率最大值为：',max(d_scores))
print('对应的最优max_depth值(可能)为：',d_scores.index(max(d_scores))+1)

In [36]:
maybeBSET_max_depth=d_scores.index(max(d_scores))+1
#使用十折交叉验证获取n_estimators（子树个数）的最优取值(可能),获取范围:1-20
n_scores = []
for i in range(1, 21):
    model = RandomForestClassifier(n_estimators= i,max_depth= maybeBSET_max_depth,criterion='entropy')
    scores = cross_val_score(model, X_source, y_source, cv=10, scoring='accuracy')
    n_scores.append(scores.mean())
print('n_estimators分别取 1~20 时得到的准确率:')
print(n_scores)
print('准确率最大值为：', max(n_scores))
print('对应的最优n_estimators值(可能)为：', n_scores.index(max(n_scores))+1)

In [37]:
#现在已经找到了max_depth与n_estimators的最优取值(可能),以此构建随机森林模型
maybeBSET_n_estimators=n_scores.index(max(n_scores))+1
'''
 sklearn建模的基本流程
1.实例化模型
2.使用fit接口将训练集带入实例化后的模型进行训练
3.使用score接口将测试集导入我们训练好的模型，去获取我们预测的分数
'''
rfc = RandomForestClassifier(n_estimators=maybeBSET_n_estimators, max_depth=maybeBSET_max_depth, criterion='entropy')
rfc = rfc.fit(X_train,y_train)
score_rfc = rfc.score(X_test,y_test)
print('Random Forest score:{}'.format(score_rfc))

In [38]:
#对kaggle提供的test.csv进行预测,并保存预测结果
X_forecast = X_all.loc[sourceRow:,:]
y_forecast = rfc.predict(X_forecast)  #这是test.csv中乘客生存的预测值
y_forecast=y_forecast.astype(int)  #因为预测值默认为浮点数,此处需要转化为整数才能符合kaggle的提交格式
passengerid = all.loc[sourceRow:,'PassengerId']
forecastDF = pd.DataFrame( { 'PassengerId': passengerid , 'Survived': y_forecast } )  #生成预测结果,以DataFrame结构表示

#查看预测结果
forecastDF.shape

In [39]:
forecastDF.head()

In [40]:
forecastDF.info()

In [41]:
#保存预测结果
forecastDF.to_csv('Aforecast.csv',index=False)